## Convolutional Neural Network - Image Classification

We have a dataset of images. These were retrieved from [SuperDataScience](https://www.superdatascience.com/deep-learning/) website. These are images of cats and dogs. We will build a Convolutional Neural Network which can classify these images. We need to pre-process these images before sending them as input to our network. But instead of doing this manually, we will be using Keras inbuilt pre-processing tools.

We have a total of 10,000 images of which 5000 are dogs and 5000 are cats. We are going to use 4000 dogs and cats in the training set and 1000 dogs and cats in the test set. Now for Keras to understand the different images and to distiguish between training set and test set, we need to create a specific folder structure as below
    * dataset/
        * training_set/
            * dogs/
            * cats/
        * test_set/
            * dogs/
            * cats/

Using this structure, the Keras library can easily label the data as cats and dogs and check for classification errors during the training phase of the algorithm. Now we are ready to build the network. 

#### Building the CNN

In [1]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
# Initialising the CNN
classifier = Sequential()

We have 4 steps in CNN. Convolution --> Pooling --> Flatten --> Full Connection

We will create the first convolution layer using Convolution2D function. If we were using video's we would need to use 3D as we also have the time parameter. 

To this layer we will specify the number of feature detectors. We will use 32 feature detectors with a dimension of 3X3 for each filter. This will be the first 3 arguments to the function. 

Next we need to specify the input_shape. We are going to use 64X64 colored images. Since all the images are not of this dimension we will force this in the image preprocessing section. After which the dimensions of the input_shape will be 64X64X3. 

The activation function we will be using is 'relu' which will help us to enforce non-linearity in the data.

In [3]:
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

Next we will perform Max Pooling on images. We will be using a pool size of 2X2. This is done to reduce the number of nodes to the next layer and also pooling detects the features regardless of its orientation introducing spatial variance.

In [4]:
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

The next step is to perform flattening. This simple process is unwrapping the matrix into a single vector which can be inputted to our fully connected layer

We will be adding a second convolutional layer to create a deeper network and also to improve the accuracy of our CNN

In [5]:
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [6]:
# Step 3 - Flattening
classifier.add(Flatten())

The final step is to add a fully connected layer. Since we have flattened the data, we can now think of the data as independent variables which can be fed to an ANN. We will have 2 layers, 1 hidden layer and 1 ouput layer. The hidden layer will have 128 nodes (This is just a random choice, no rule of thumb) and output layer will have just 1 node as we are using binary classification.

In [7]:
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

Now all our layers are created, we need to compile the network. In this process we will be fixing the parameters of the netwrok.

optimizer - Algorithm we want to use to find the optimal set of weight for the network. We will be using stochastic gradient descent for this problem. 'adam' is the most common SGD algorithm.

loss - We need to fix a loss function within the SGD algorithm. We wil be 'binary_crossentropy' which is similar to the loss funciton in logistic regression model. If we have more than 2 categories in the output, we need to use 'categorical_crossentropy'.

metrics - A measure to find the performance of the model.

In [8]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Now our network is ready. We can send in images to be classified. But as discussed earlier, we need to pre-process the images before sending them to the network

#### Fitting the images to the network

We will be performing some image augmentation before training. This is done to prevent overfitting. Also, we have just 8000 images to train the network. This is not a lot of images and we might not get good accuracy. Keras will create multiple batches of the image and to each batch it will apply some random transformation to the image such as rotating, flipping, shifting, sheering etc. 

In [9]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


Now that we have the training and the test sets, we can train our network.

In [ ]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 8000,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 2000)

* Epoch 1/25
250/250 [==============================] - 304s 1s/step - loss: 0.6511 - acc: 0.6118 - val_loss: 0.5887 - val_acc: 0.6768
* Epoch 2/25
250/250 [==============================] - 308s 1s/step - loss: 0.5861 - acc: 0.6944 - val_loss: 0.6040 - val_acc: 0.6763
* Epoch 3/25
250/250 [==============================] - 324s 1s/step - loss: 0.5505 - acc: 0.7208 - val_loss: 0.5236 - val_acc: 0.7464
* Epoch 4/25
250/250 [==============================] - 334s 1s/step - loss: 0.5196 - acc: 0.7416 - val_loss: 0.5326 - val_acc: 0.7379
* Epoch 5/25
250/250 [==============================] - 306s 1s/step - loss: 0.4887 - acc: 0.7584 - val_loss: 0.4805 - val_acc: 0.7680
* Epoch 6/25
250/250 [==============================] - 308s 1s/step - loss: 0.4873 - acc: 0.7649 - val_loss: 0.4675 - val_acc: 0.7785
* Epoch 7/25
250/250 [==============================] - 300s 1s/step - loss: 0.4643 - acc: 0.7719 - val_loss: 0.4769 - val_acc: 0.7799
* Epoch 8/25
250/250 [==============================] - 300s 1s/step - loss: 0.4445 - acc: 0.7925 - val_loss: 0.4559 - val_acc: 0.7927
* Epoch 9/25
250/250 [==============================] - 301s 1s/step - loss: 0.4401 - acc: 0.7959 - val_loss: 0.4388 - val_acc: 0.7954
* Epoch 10/25
250/250 [==============================] - 299s 1s/step - loss: 0.4221 - acc: 0.8019 - val_loss: 0.4669 - val_acc: 0.7800
* Epoch 11/25
250/250 [==============================] - 324s 1s/step - loss: 0.4134 - acc: 0.8114 - val_loss: 0.4567 - val_acc: 0.7889
* Epoch 12/25
250/250 [==============================] - 469s 2s/step - loss: 0.3959 - acc: 0.8195 - val_loss: 0.4568 - val_acc: 0.7869
* Epoch 13/25
250/250 [==============================] - 371s 1s/step - loss: 0.3915 - acc: 0.8196 - val_loss: 0.4405 - val_acc: 0.7969
* Epoch 14/25
250/250 [==============================] - 395s 2s/step - loss: 0.3840 - acc: 0.8283 - val_loss: 0.4531 - val_acc: 0.7869
* Epoch 15/25
250/250 [==============================] - 331s 1s/step - loss: 0.3700 - acc: 0.8320 - val_loss: 0.4411 - val_acc: 0.8131
* Epoch 16/25
250/250 [==============================] - 283s 1s/step - loss: 0.3695 - acc: 0.8269 - val_loss: 0.4332 - val_acc: 0.8119
* Epoch 17/25
250/250 [==============================] - 387s 2s/step - loss: 0.3518 - acc: 0.8420 - val_loss: 0.4239 - val_acc: 0.8100
* Epoch 18/25
250/250 [==============================] - 362s 1s/step - loss: 0.3428 - acc: 0.8442 - val_loss: 0.4261 - val_acc: 0.8150
* Epoch 19/25
250/250 [==============================] - 390s 2s/step - loss: 0.3254 - acc: 0.8596 - val_loss: 0.4212 - val_acc: 0.8255
* Epoch 20/25
250/250 [==============================] - 399s 2s/step - loss: 0.3150 - acc: 0.8644 - val_loss: 0.4395 - val_acc: 0.8154
* Epoch 21/25
250/250 [==============================] - 339s 1s/step - loss: 0.3089 - acc: 0.8635 - val_loss: 0.4380 - val_acc: 0.8180
* Epoch 22/25
250/250 [==============================] - 361s 1s/step - loss: 0.3020 - acc: 0.8679 - val_loss: 0.4338 - val_acc: 0.8131
* Epoch 23/25
250/250 [==============================] - 323s 1s/step - loss: 0.2916 - acc: 0.8760 - val_loss: 0.4246 - val_acc: 0.8195
* Epoch 24/25
250/250 [==============================] - 308s 1s/step - loss: 0.2846 - acc: 0.8785 - val_loss: 0.4684 - val_acc: 0.8125
* Epoch 25/25
250/250 [==============================] - 303s 1s/step - loss: 0.2717 - acc: 0.8830 - val_loss: 0.4480 - val_acc: 0.8180